In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Set up the notebook environment
!pip install catboost xgboost lightgbm scikit-learn
import catboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy.io
from scipy.io import savemat
from scipy import stats
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.signal import butter, filtfilt, welch, decimate
from scipy.interpolate import CubicSpline
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from scipy.io import loadmat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 5.7 MB/s eta 0:00:00


In [ ]:
data_new_1 = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/additional data/BCICIV_4_mat/sub1_comp.mat')
data_new_2 = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/additional data/BCICIV_4_mat/sub2_comp.mat')
data_new_3 = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/additional data/BCICIV_4_mat/sub3_comp.mat')

In [ ]:
data_train = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/Ted Yang/Copy of raw_training_data.mat')
data_test = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/Ted Yang/Copy of leaderboard_data.mat')

In [ ]:
data_glove = data_train['train_dg']
data_ecog = data_train['train_ecog']
test_ecog = data_test['leaderboard_ecog']

train_dg = []
train_ecog = []

# Function to load the entire dataset as training data
def load_data(data):
    return data[0]  # Assuming each 'data' item is wrapped in an outer array

# Loop over each subject's data for glove and ECoG data
for glove_data, ecog_data in zip(data_glove, data_ecog):
    train_dg.append(load_data(glove_data))
    train_ecog.append(load_data(ecog_data))

# Now train_dg and train_ecog contain all the data for each subject as training data

# Calculate total number of samples in the training set for the first subject (as an example)
total_samples_train_ecog = len(train_ecog[0])

print(f'The total number of samples in the training set is {total_samples_train_ecog}.')


The total number of samples in the training set is 300000.


In [ ]:
ecog_test = []

for ecog_t in test_ecog:
    ecog_test.append(load_data(ecog_t))

total_samples_test_ecog = len(ecog_test[0])

print(total_samples_test_ecog)

147500


In [ ]:
def filter_data(raw_eeg, fs=1000):
    """
    Input:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)

    Output:
      clean_data (samples x channels): the filtered signal
    """

    #lowcut and highcut for the bandpass filter
    lowcut = 0.15
    highcut = 200.0

    #Butterworth bandpass filter
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(4, [low, high], btype='band', analog=False)

    #apply filter to raw ECoG data
    clean_data = filtfilt(b, a, raw_eeg, axis=0)

    return clean_data


In [ ]:
#window and step sizes
window_length = 100
step_size = 50

#number of feature windows M for the full dataset, training, and testing sets
#M_full = 1 + (total_samples_full_ecog - window_length) // step_size
M_train = 1 + (total_samples_train_ecog - window_length) // step_size
#M_test = 1 + (total_samples_test_ecog - window_length) // step_size

#print(f'There are {M_full} windows using all the data.')
print(f'There are {M_train} windows training data.')
#print(f'There are {M_test} windows testing data.')

There are 5999 windows training data.


In [ ]:
def get_features(filtered_window, fs=1000):
    """
    Inputs:
        filtered_window (window_samples x channels): the window of the filtered ECoG signal
        fs: sampling rate

    Output:
        features (channels x num_features): the features calculated on each channel for the window
    """
    # Make sure nperseg does not exceed window size
    nperseg = min(filtered_window.shape[0], 256)

    # Basic statistical features
    mean = np.mean(filtered_window, axis=0)
    variance = np.var(filtered_window, axis=0)
    lmp = np.mean(np.abs(filtered_window), axis=0)  # Local Motor Potential

    # Compute the Power Spectral Density (PSD) for each channel
    freqs, psd = welch(filtered_window.T, fs, nperseg=nperseg)

    # Define frequency bands
    bands = {
        'alpha': (8, 12),
        'beta': (13, 30),
        'theta': (4, 7),  # Adding an extra band as an example
        #'low_gamma': (30, 50),  # Another example band
        'custom_band1': (18, 24),
        'custom_band2': (75, 115),
        'custom_band3': (125, 159),
        'custom_band4': (159, 175)
    }

    # Initialize a dictionary to hold the power for each band
    band_powers = {band: np.zeros(psd.shape[0]) for band in bands}

    # Calculate power for each band
    for band, (low, high) in bands.items():
        freq_mask = (freqs >= low) & (freqs <= high)
        band_powers[band] = np.sum(psd[:, freq_mask], axis=1)

    # Collect all features
    all_features = [mean, variance, lmp] + list(band_powers.values())
    features = np.vstack(all_features).T

    return features


In [ ]:
def get_windowed_feats(raw_ecog, fs, window_length_ms, window_overlap_ms):
    """
    Inputs:
        raw_ecog (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
        window_length_ms: the window's length in milliseconds
        window_overlap_ms: the window's overlap in milliseconds

    Output:
        all_feats (num_windows x (channels x features)): the features for each channel for each time window
    """
    window_length = int(fs * (window_length_ms / 1000))
    window_overlap = int(fs * (window_overlap_ms / 1000))
    step_size = window_length - window_overlap

    #filter_data function placeholder
    filtered_ecog = raw_ecog

    total_samples = raw_ecog.shape[0]
    num_windows = 1 + (total_samples - window_length) // step_size

    feature_list = []

    for start_idx in tqdm(range(0, total_samples - window_length + 1, step_size)):
        end_idx = start_idx + window_length
        window = filtered_ecog[start_idx:end_idx, :]

        #features for this window
        window_features = get_features(window, fs)

        #append features to list
        feature_list.append(window_features.flatten())

    #stack to get 2D array (num_windows x features)
    all_feats = np.array(feature_list)

    return all_feats

In [ ]:
def create_R_matrix(features, N=3):
    """
    Input:
        features: The 2D array of features (num_windows x num_features_per_window).
        N: The number of previous windows to include for each prediction instance.

    Output:
        R: The response matrix.
    """
    #first N-1 rows to beginning of features matrix
    adjusted_features = np.vstack([features[:N-1], features])

    #number of total instances after adjustment
    M_prime = adjusted_features.shape[0] - (N - 1)

    #initialize response matrix R
    num_features = features.shape[1]
    R = np.zeros((M_prime, N * num_features + 1))  # +1 for the intercept term

    #fill in R
    for i in range(M_prime):
        #extract N consecutive windows of features
        consecutive_features = adjusted_features[i:i+N].flatten()
        #fill corresponding row in R, adding 1 as the last column
        R[i, 1:] = consecutive_features
        R[i, 0] = 1  # Intercept term

    return R

In [ ]:
train_ecog_1 = train_ecog[0]
train_ecog_2 = train_ecog[1]
train_ecog_3 = train_ecog[2]

train_glove_1 = train_dg[0]
train_glove_2 = train_dg[1]
train_glove_3 = train_dg[2]

In [ ]:
#feature extraction for each subject
feature_1 = get_windowed_feats(train_ecog_1, 1000, 100, 50)
feature_2 = get_windowed_feats(train_ecog_2, 1000, 100, 50)
feature_3 = get_windowed_feats(train_ecog_3, 1000, 100, 50)

num_target_windows = feature_1.shape[0]
num_target_windows

100%|██████████| 5999/5999 [00:12<00:00, 462.86it/s]


5999

In [ ]:
#get R matrix
R_1 = create_R_matrix(feature_1, N=3)
R_2 = create_R_matrix(feature_2, N=3)
R_3 = create_R_matrix(feature_3, N=3)

In [ ]:
def get_windowed_target(glove_data, window_length_ms, window_overlap_ms, fs=1000):
    window_length = int(fs * (window_length_ms / 1000))
    window_overlap = int(fs * (window_overlap_ms / 1000))
    step_size = window_length - window_overlap

    num_windows = 1 + (glove_data.shape[0] - window_length) // step_size
    targets = []

    for start_idx in range(0, glove_data.shape[0] - window_length + 1, step_size):
        end_idx = start_idx + window_length
        window = glove_data[start_idx:end_idx, :]
        targets.append(np.mean(window, axis=0))  # Example: mean over the window

    return np.array(targets)

# Windowing target glove data
target_1 = get_windowed_target(train_glove_1, 100, 50)
target_2 = get_windowed_target(train_glove_2, 100, 50)
target_3 = get_windowed_target(train_glove_3, 100, 50)


In [ ]:
#LGBoost
num_leaves = 2**5

lgb_model1 = [LGBMRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, num_leaves=num_leaves, force_col_wise=True) for _ in range(target_1.shape[1])]
for i, model in enumerate(lgb_model1):
    model.fit(R_1[:, 1:], target_1[:, i])

lgb_model2 = [LGBMRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, num_leaves=num_leaves, force_col_wise=True) for _ in range(target_2.shape[1])]
for i, model in enumerate(lgb_model2):
    model.fit(R_2[:, 1:], target_2[:, i])

lgb_model3 = [LGBMRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, num_leaves=num_leaves, force_col_wise=True) for _ in range(target_3.shape[1])]
for i, model in enumerate(lgb_model3):
    model.fit(R_3[:, 1:], target_3[:, i])

In [ ]:
num_leaves = 2**5

lgb_multioutput1 = MultiOutputRegressor(LGBMRegressor(n_estimators=300, max_depth=5, learning_rate=0.1, num_leaves=num_leaves, force_col_wise=True))
lgb_multioutput1.fit(R_1[:, 1:], target_1)

lgb_multioutput2 = MultiOutputRegressor(LGBMRegressor(n_estimators=300, max_depth=5, learning_rate=0.1, num_leaves=num_leaves, force_col_wise=True))
lgb_multioutput2.fit(R_2[:, 1:], target_2)

lgb_multioutput3 = MultiOutputRegressor(LGBMRegressor(n_estimators=300, max_depth=5, learning_rate=0.1, num_leaves=num_leaves, force_col_wise=True))
lgb_multioutput3.fit(R_3[:, 1:], target_3)

[LightGBM] [Info] Total Bins 426870
[LightGBM] [Info] Number of data points in the train set: 5999, number of used features: 1674
[LightGBM] [Info] Start training from score -0.031320
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

MultiOutputRegressor(estimator=LGBMRegressor(force_col_wise=True, max_depth=5,
                                             n_estimators=300, num_leaves=32))

In [ ]:
# XGBoost Model
xgb_model1 = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1)
xgb_model1.fit(R_1[:, 1:], target_1)

xgb_model2 = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1)
xgb_model2.fit(R_2[:, 1:], target_2)

xgb_model3 = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1)
xgb_model3.fit(R_3[:, 1:], target_3)

# CatBoost Model
cat_model1 = CatBoostRegressor(iterations=300, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model1.fit(R_1[:, 1:], target_1)

cat_model2 = CatBoostRegressor(iterations=300, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model2.fit(R_2[:, 1:], target_2)

cat_model3 = CatBoostRegressor(iterations=300, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model3.fit(R_3[:, 1:], target_3)

In [ ]:
def linear_filter(R_matrix, train_data, lambda_reg=0.01):
    num_samples_features = R_matrix.shape[0]
    num_samples_flexion = train_data.shape[0]

    # Downsample train_data if it doesn't match the number of feature windows in R_matrix
    if num_samples_features != num_samples_flexion:
        downsample_factor = num_samples_flexion // num_samples_features
        downsample_factor = max(downsample_factor, 1)  # Ensure downsample_factor at least 1
        downsize_data = decimate(train_data, downsample_factor, axis=0)

        # Ensure downsize_data has same number of samples as R_matrix
        if downsize_data.shape[0] > num_samples_features:
            downsize_data = downsize_data[:num_samples_features]
    else:
        downsize_data = train_data

    # Regularized linear filter f using matrix multiplication
    # p1 = (R^T R + lambda * I)^(-1)
    lambda_identity = lambda_reg * np.eye(R_matrix.shape[1])  # Lambda times identity matrix
    p1 = np.linalg.inv(np.matmul(R_matrix.T, R_matrix) + lambda_identity)

    # p2 = R^T Y
    p2 = np.matmul(R_matrix.T, downsize_data)

    # f = p1 * p2
    lf = np.matmul(p1, p2)

    return downsize_data, lf

In [ ]:
test_ecog_1 = ecog_test[0]
test_ecog_2 = ecog_test[1]
test_ecog_3 = ecog_test[2]

In [ ]:
#feature extraction of test set
test_feature_1 = get_windowed_feats(test_ecog_1, 1000, 100, 50)
test_feature_2 = get_windowed_feats(test_ecog_2, 1000, 100, 50)
test_feature_3 = get_windowed_feats(test_ecog_3, 1000, 100, 50)

100%|██████████| 2949/2949 [00:02<00:00, 1001.34it/s]


In [ ]:
#R matrix for test set
test_R_1 = create_R_matrix(test_feature_1, N=3)
test_R_2 = create_R_matrix(test_feature_2, N=3)
test_R_3 = create_R_matrix(test_feature_3, N=3)

In [ ]:
xgb_preds_1 = xgb_model1.predict(test_R_1[:, 1:])
xgb_preds_2 = xgb_model2.predict(test_R_2[:, 1:])
xgb_preds_3 = xgb_model3.predict(test_R_3[:, 1:])

cat_preds_1 = cat_model1.predict(test_R_1[:, 1:])
cat_preds_2 = cat_model2.predict(test_R_2[:, 1:])
cat_preds_3 = cat_model3.predict(test_R_3[:, 1:])

In [ ]:
lgb_preds_1 = lgb_model1.predict(test_R_1[:, 1:])
lgb_preds_2 = lgb_model2.predict(test_R_2[:, 1:])
lgb_preds_3 = lgb_model3.predict(test_R_3[:, 1:])

In [ ]:
multi_lgb_preds_1 = lgb_multioutput1.predict(test_R_1[:, 1:])
multi_lgb_preds_2 = lgb_multioutput2.predict(test_R_2[:, 1:])
multi_lgb_preds_3 = lgb_multioutput3.predict(test_R_3[:, 1:])

In [ ]:
xgb_preds_1, xgb_preds_2, xgb_preds_3

(array([[-0.2866884 ,  0.22059734, -0.1880851 , -0.71982867, -0.26083305],
        [-0.46558037, -0.32097867, -0.29731467, -0.82368916, -0.22325626],
        [-0.24004364,  0.19402324, -0.23935698, -0.7049758 , -0.11601047],
        ...,
        [-0.36768886, -0.16867968,  0.00983223,  1.3170851 ,  0.22237681],
        [-0.13134933,  0.51806307,  0.58514   ,  1.7544448 ,  1.2303607 ],
        [-0.5043937 ,  0.06531288, -0.07332338,  0.5123098 ,  0.09671648]],
       dtype=float32),
 array([[-0.20364623, -0.30127344, -0.02308545, -0.4068105 , -0.18695658],
        [ 0.10534757, -0.06317123, -0.1045859 , -0.42087132, -0.25476453],
        [-0.02368988, -0.30219814,  0.10271062, -0.41317368, -0.14640905],
        ...,
        [-0.05734504,  1.2693515 ,  0.8300764 ,  1.0394187 , -0.05468125],
        [-0.48340428, -0.26398733,  0.5579079 ,  1.107155  , -0.17349707],
        [-0.6921831 , -0.2612423 ,  0.9480626 ,  1.1302867 ,  0.3044645 ]],
       dtype=float32),
 array([[-0.3408563 ,  0.0

In [ ]:
cat_preds_1, cat_preds_2, cat_preds_3

(array([[-0.14279201,  0.30398801, -0.29644186, -0.62382088, -0.2630482 ],
        [-0.16083369,  0.02164009, -0.39194016, -0.6446817 , -0.31050994],
        [-0.19684014, -0.04872954, -0.296318  , -0.62347438, -0.26055692],
        ...,
        [-0.36195861,  0.21389847, -0.05632899,  0.68227836,  1.09015424],
        [-0.52647675, -0.04665038,  0.18744346,  1.4661065 ,  0.51400037],
        [-0.56397341,  0.24568295,  0.09788081,  1.60147515,  0.29330736]]),
 array([[-0.02741646, -0.23282886, -0.18898204, -0.40718104, -0.07230983],
        [ 0.01205423, -0.08054634, -0.24923251, -0.42503724,  0.02692769],
        [ 0.08039947, -0.1051417 , -0.08487174, -0.35398881,  0.05789329],
        ...,
        [ 0.21244496,  0.6677167 ,  0.37635169,  0.18164351, -0.04469066],
        [-0.69990509,  0.21799576,  0.53896397,  1.05953965,  0.09313872],
        [-0.76486848,  0.10313067,  0.43342818,  1.01782418,  0.483202  ]]),
 array([[-0.24452664,  0.19480634,  0.03279445,  0.51792638,  0.060875

In [ ]:
lgb_preds_1, lgb_preds_3, lgb_preds_3

In [ ]:
multi_lgb_preds_1, multi_lgb_preds_2, multi_lgb_preds_3

(array([[-0.2830769 ,  0.06263234, -0.36524769, -0.73814458, -0.39119266],
        [-0.30127855, -0.07503009, -0.16451314, -0.69536254, -0.41757492],
        [-0.35697377, -0.05902427, -0.47166692, -0.70463198, -0.24808925],
        ...,
        [-0.36477321,  0.06877124, -0.17521623,  0.75172509,  0.50822907],
        [-0.36817197,  0.283439  ,  0.144693  ,  1.0289049 ,  0.66819434],
        [-0.12328453,  0.01081282, -0.16385547,  0.57435511,  0.23084888]]),
 array([[-0.08960865, -0.20999042, -0.14523129, -0.12747283, -0.20779115],
        [ 0.09895697, -0.25437788, -0.3086287 , -0.36355443, -0.32248796],
        [ 0.22295792,  0.04447895,  0.04451524, -0.18358244,  0.00474366],
        ...,
        [-0.59233592,  0.90570284,  0.37773302,  0.44054649,  0.11175192],
        [-0.6947237 , -0.21453885,  0.34856442,  1.56776524,  0.07486307],
        [-0.63230921, -0.00952924,  0.2089814 ,  1.25316209,  0.83149038]]),
 array([[-0.33253824, -0.10054125, -0.44596899,  0.53856943, -0.037308

In [ ]:
from scipy.io import savemat

In [ ]:
original_length = xgb_preds_1.shape[0]
desired_length = test_ecog_1.shape[0]

In [ ]:
def interpolate(prediction, original_length, desired_length):
    # Step 1: Create a time array for the original prediction
    original_time = np.linspace(0, 1, original_length)

    # Step 2: Create a time array for the desired length
    desired_time = np.linspace(0, 1, desired_length)

    # Step 3: Interpolate each column separately using cubic spline
    interpolated_prediction = np.zeros((desired_length, prediction.shape[1]))
    for i in range(prediction.shape[1]):
        cs = CubicSpline(original_time, prediction[:, i])
        interpolated_prediction[:, i] = cs(desired_time)

    return interpolated_prediction

## **XGBoost**

**Training Correaltion 1.0**

([0.42975486277270897,
  0.576615967656113,
  0.14138890739763021,
  0.44750700729701787,
  0.10267421165647812],
 [0.5077436689148577,
  0.18126293860768045,
  0.21879286202665796,
  0.5181947857825426,
  0.07912924578167839],
 [0.5731551697779543,
  0.3824455186605641,
  0.46025726563488767,
  0.5024670634360416,
  0.37533030506974796])

**Training Correaltion 2.0**

([0.4100421857991916,
  0.5743259992127828,
  0.14748974360928555,
  0.44677582770641555,
  0.10947379284793216],
 [0.47727048144531226,
  0.1871533408946265,
  0.21751776062477465,
  0.5070114060467675,
  0.05319920776450609],
 [0.574777095515358,
  0.3847420304645163,
  0.47489104204454585,
  0.5214911516355769,
  0.38291076934580537])

In [ ]:
inter_xgb_prediction1 = interpolate(xgb_preds_1, original_length, desired_length)
inter_xgb_prediction2 = interpolate(xgb_preds_2, original_length, desired_length)
inter_xgb_prediction3 = interpolate(xgb_preds_3, original_length, desired_length)

In [ ]:
#XGBoost Submission
predicted_dg = np.zeros((3,1), dtype=object)
predicted_dg[0,0] = inter_xgb_prediction1
predicted_dg[1,0] = inter_xgb_prediction2
predicted_dg[2,0] = inter_xgb_prediction3

savemat('predictions.mat', {'predicted_dg':predicted_dg})

In [ ]:
print(predicted_dg)

[[array([[-0.28668839,  0.22059734, -0.18808509, -0.71982867, -0.26083305],
         [-0.2965839 ,  0.17642866, -0.18778146, -0.72890276, -0.2650773 ],
         [-0.30617497,  0.13407484, -0.18766082, -0.73759611, -0.2690325 ],
         ...,
         [-0.47050925,  0.15290626,  0.00274289,  0.60171549,  0.24792825],
         [-0.48721609,  0.11008089, -0.03463673,  0.55746563,  0.17376196],
         [-0.5043937 ,  0.06531288, -0.07332338,  0.51230979,  0.09671648]])]
 [array([[-0.20364623, -0.30127344, -0.02308545, -0.40681049, -0.18695658],
         [-0.18489828, -0.28069535, -0.03523009, -0.40866164, -0.19094436],
         [-0.16682693, -0.26098295, -0.04679378, -0.41042191, -0.19480682],
         ...,
         [-0.69310992, -0.37144214,  0.90094673,  1.11523279,  0.25990418],
         [-0.69276958, -0.31790777,  0.92407865,  1.12253467,  0.28184648],
         [-0.69218308, -0.2612423 ,  0.9480626 ,  1.13028669,  0.30446449]])]
 [array([[-0.34085631,  0.00145504, -0.28401414,  0.5933

## **CatBoost**

**Training Correaltion 1.0: n_estimators = 100**

([0.42110667057890294,
  0.5955357296984822,
  0.16680256550611047,
  0.49621501159105874,
  0.09428410688813813],
 [0.47397291001606884,
  0.18774692153335576,
  0.28783016125629185,
  0.4229055985315709,
  0.3153480826008141],
 [0.5941576953268893,
  0.3906184545554212,
  0.4958015312549799,
  0.5647895824977968,
  0.42089092458207944])

**Training Correaltion 2.0:**

([0.44241796425362817,
  0.59141093373409,
  0.17263839404211603,
  0.5097063215019813,
  0.08280680860252813],
  
  [0.4797022539095192,
  0.22737009439038464,
  0.2948164169286338,
  0.43781216948166124,
  0.31511744282831045],
 [0.5788678488123089,
  0.38464896870682586,
  0.4874284847984901,
  0.5415574332727642,
  0.4085850945940047])

**Training Correaltion 3.0: n_estimators = 300**

([0.4515281141722916,
  0.6100341432652044,
  0.17987936346454056,
  0.49870862185773174,
  0.11510920928297969],
 [0.49312217446906437,
  0.19973550922066757,
  0.28184504641469527,
  0.4870203260072296,
  0.33360122641518797],
 [0.5867612865384663,
  0.38391599391965514,
  0.516004225557485,
  0.5726448589428662,
  0.43779103958155086])

**0.4645 --> Training Correaltion 4.0: n_estimators = 500** 0.4109

([0.45915435220582007,
  0.6227743996948135,
  0.18534008240262018,
  0.49759615378040606,
  0.11338631133452018],
 [0.49761276929231835,
  0.20979752269548407,
  0.28649344820015876,
  0.49269942939590744,
  0.316072076934109],
 [0.5906187000936859,
  0.38211854173786647,
  0.5084406165458811,
  0.5727193489351534,
  0.43008815127292777])

In [ ]:
inter_cat_prediction1 = interpolate(cat_preds_1, original_length, desired_length)
inter_cat_prediction2 = interpolate(cat_preds_2, original_length, desired_length)
inter_cat_prediction3 = interpolate(cat_preds_3, original_length, desired_length)

In [ ]:
#CatBoost Submission
predicted_dg = np.zeros((3,1), dtype=object)
predicted_dg[0,0] = inter_cat_prediction1
predicted_dg[1,0] = inter_cat_prediction2
predicted_dg[2,0] = inter_cat_prediction3

savemat('predictions.mat', {'predicted_dg':predicted_dg})

In [ ]:
predicted_dg

array([[array([[-0.14279201,  0.30398801, -0.29644186, -0.62382088, -0.2630482 ],
               [-0.14342239,  0.29168919, -0.30201206, -0.61947001, -0.26604773],
               [-0.14403295,  0.27975242, -0.30739751, -0.61541604, -0.26894192],
               ...,
               [-0.57619566,  0.20206766,  0.11876128,  1.6408195 ,  0.2637212 ],
               [-0.57028609,  0.22343196,  0.10855789,  1.62178138,  0.27795007],
               [-0.56397341,  0.24568295,  0.09788081,  1.60147515,  0.29330736]])],
       [array([[-0.02741646, -0.23282886, -0.18898204, -0.40718104, -0.07230983],
               [-0.02826931, -0.22539021, -0.1930383 , -0.40982233, -0.06958485],
               [-0.02902832, -0.21818349, -0.19696533, -0.41234248, -0.06689155],
               ...,
               [-0.82367786,  0.08813761,  0.46035288,  1.0665395 ,  0.45192785],
               [-0.79514734,  0.09536178,  0.44721829,  1.04282461,  0.46734526],
               [-0.76486848,  0.10313067,  0.43342818, 

## **LGBoost**

Multi

In [ ]:
inter_multi_lgb_preds_1 = interpolate(multi_lgb_preds_1, original_length, desired_length)
inter_multi_lgb_preds_2 = interpolate(multi_lgb_preds_2, original_length, desired_length)
inter_multi_lgb_preds_3 = interpolate(multi_lgb_preds_3, original_length, desired_length)

In [ ]:
#LGBoost Submission
predicted_dg = np.zeros((3,1), dtype=object)
predicted_dg[0,0] = inter_multi_lgb_preds_1
predicted_dg[1,0] = inter_multi_lgb_preds_2
predicted_dg[2,0] = inter_multi_lgb_preds_3

#savemat('predictions.mat', {'predicted_dg':predicted_dg})

In [ ]:
predicted_dg

In [ ]:
inter_lgb_prediction1 = interpolate(lgb_preds_1, original_length, desired_length)
inter_lgb_prediction2 = interpolate(lgb_preds_2, original_length, desired_length)
inter_lgb_prediction3 = interpolate(lgb_preds_3, original_length, desired_length)

In [ ]:
#LGBoost Submission
predicted_dg = np.zeros((3,1), dtype=object)
predicted_dg[0,0] = inter_lgb_prediction1
predicted_dg[1,0] = inter_lgb_prediction2
predicted_dg[2,0] = inter_lgb_prediction3

savemat('predictions.mat', {'predicted_dg':predicted_dg})

In [ ]:
predicted_dg



---



In [ ]:
pred = loadmat('predictions.mat')
print(pred['predicted_dg'])


[[array([[-0.14279201,  0.30398801, -0.29644186, -0.62382088, -0.2630482 ],
         [-0.14342239,  0.29168919, -0.30201206, -0.61947001, -0.26604773],
         [-0.14403295,  0.27975242, -0.30739751, -0.61541604, -0.26894192],
         ...,
         [-0.57619566,  0.20206766,  0.11876128,  1.6408195 ,  0.2637212 ],
         [-0.57028609,  0.22343196,  0.10855789,  1.62178138,  0.27795007],
         [-0.56397341,  0.24568295,  0.09788081,  1.60147515,  0.29330736]])]
 [array([[-0.02741646, -0.23282886, -0.18898204, -0.40718104, -0.07230983],
         [-0.02826931, -0.22539021, -0.1930383 , -0.40982233, -0.06958485],
         [-0.02902832, -0.21818349, -0.19696533, -0.41234248, -0.06689155],
         ...,
         [-0.82367786,  0.08813761,  0.46035288,  1.0665395 ,  0.45192785],
         [-0.79514734,  0.09536178,  0.44721829,  1.04282461,  0.46734526],
         [-0.76486848,  0.10313067,  0.43342818,  1.01782418,  0.483202  ]])]
 [array([[-0.24452664,  0.19480634,  0.03279445,  0.5179